# Install and setup kaggle and api key
get a kaggle account
Get Kaggle API
conda install kaggle
copy the kaggle.json file from download to your user directory/.kaggle


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import kaggle

In [ ]:
# Get the data using an API call
kaggle.api.dataset_download_files('rodsaldanha/arketing-campaign', path='resources', unzip=True)

In [ ]:
# Import the data
data = pd.read_csv("./resources/marketing_campaign.csv",delimiter=';')


# EDA (Exploratory Data Analysis)
We will revisit this. For now We want the rough draft of the model
#
During EDA

Visualize the data using plots and graphs to understand distributions and relationships between variables.
Calculate summary statistics to get a sense of the central tendencies and variability.
Identify any correlations between variables that might influence model choices.
Detect and treat missing values or outliers that could skew the results of your analysis.
Explore the data's structure to inform feature selection and engineering, which are key to building effective machine learning models.


In [ ]:
display (data.head())
# what does our data look like? At this point also use any documentation on the data set to find out what each value means and how it might be used is solving the business problem
print (f'Columns with NA valuses \n {data.isna().sum()[lambda x: x > 0]}')
# Make desision about null values. Can we fill them of should we drop rows with null values?
non_numeric= (data.dtypes[(data.dtypes != 'int64') & (data.dtypes != 'float64')]).index.tolist()
print (f'Columns that are not numeric :\n {non_numeric}')
# Explore non numberic type to see how we can use them in the model

In [ ]:
# just to get started we will drop NA and columns that are not numberic. this will let us get a rough model
# we come back to this and preprocess based on the draft results if needed


data_drop_columns = data.drop(columns=non_numeric, axis=1)
data_drop_na = data_drop_columns.dropna()
df = data_drop_na.copy()
df.head()



In [ ]:
# Split data into Train and Test
X = df.drop('Response', axis=1)
y = df["Response"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
# This will split 'X' and 'y' such that 80% is used for training and 20% is used for testing.

what does a logistic model score look like without scaling?

# Scaling the data evens out the high and low values so on one column dominates over all others
We will want to compare the scores of standard scalar to Min Max scalar to pick the bast scaling methood.

In [ ]:
# Scale the X data by using StandardScaler()
scaler_ss = StandardScaler().fit(X_train)
X_train_ss_scaled = scaler_ss.transform(X_train)
X_train_ss_scaled

# Transform the test dataset based on the fit from the training dataset
X_test_ss_scaled = scaler_ss.transform(X_test)
X_test_ss_scaled

In [ ]:
# now lets look at min max scaler
scaler_mm = MinMaxScaler().fit(X_train)
X_train_mm_scaled = scaler_mm.transform(X_train)
display (X_train_mm_scaled)

X_test_mm_scaled = scaler_mm.transform(X_test)
display (X_test_mm_scaled)

In [ ]:
# Create a `LogisticRegression` function and assign it 
# to a variable named `logistic_regression_model`.
#
logistic_regression_model_ss = LogisticRegression()
# Fit the model
logistic_regression_model_ss.fit(X_train_ss_scaled, y_train)

logistic_regression_model_mm = LogisticRegression()

logistic_regression_model_mm.fit(X_train_mm_scaled, y_train)


In [ ]:
# Score the model

print(f"Standard Scaler\nTraining Data Score: {logistic_regression_model_ss.score(X_train_ss_scaled, y_train)}")
print(f"Testing Data Score: {logistic_regression_model_ss.score(X_test_ss_scaled, y_test)}")
print(f"Min Max Scaler\nTraining Data Score: {logistic_regression_model_mm.score(X_train_mm_scaled, y_train)}")
print(f"Testing Data Score: {logistic_regression_model_mm.score(X_test_mm_scaled, y_test)}")

# Romario models

In [ ]:
# **RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier

# Create and train the model
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"Random Forest - Training Data Score: {random_forest_model.score(X_train_ss_scaled, y_train)}")
print(f"Random Forest - Testing Data Score: {random_forest_model.score(X_test_ss_scaled, y_test)}")


In [ ]:
# *******RANDOM FOREST WITH METRICS*********

# Predict on test set
y_pred = random_forest_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"Random Forest - Precision: {precision_score(y_test, y_pred)}")
print(f"Random Forest - Recall: {recall_score(y_test, y_pred)}")
print(f"Random Forest - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(random_forest_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"Random Forest - Cross-Validation Accuracy: {cv_scores.mean()}")


In [ ]:
# ******* GradientBoostingClassifier MODELING
from sklearn.ensemble import GradientBoostingClassifier

# Create and train the model
gbm_model = GradientBoostingClassifier(random_state=42)
gbm_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"Gradient Boosting Machine - Training Data Score: {gbm_model.score(X_train_ss_scaled, y_train)}")
print(f"Gradient Boosting Machine - Testing Data Score: {gbm_model.score(X_test_ss_scaled, y_test)}")


In [ ]:
# ******* GradientBoostingClassifier MODELING WITH METRICS

# Create and train the model
gbm_model = GradientBoostingClassifier(random_state=1)
gbm_model.fit(X_train_ss_scaled, y_train)

# Predict on test set
y_pred = gbm_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"Gradient Boosting Machine - Precision: {precision_score(y_test, y_pred)}")
print(f"Gradient Boosting Machine - Recall: {recall_score(y_test, y_pred)}")
print(f"Gradient Boosting Machine - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(gbm_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"Gradient Boosting Machine - Cross-Validation Accuracy: {cv_scores.mean()}")


In [ ]:


# Create and train the model
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"K-Nearest Neighbors - Training Data Score: {knn_model.score(X_train_ss_scaled, y_train)}")
print(f"K-Nearest Neighbors - Testing Data Score: {knn_model.score(X_test_ss_scaled, y_test)}")


In [ ]:


# Create and train the model
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_ss_scaled, y_train)

# Predict on test set
y_pred = knn_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"K-Nearest Neighbors - Precision: {precision_score(y_test, y_pred)}")
print(f"K-Nearest Neighbors - Recall: {recall_score(y_test, y_pred)}")
print(f"K-Nearest Neighbors - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(knn_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"K-Nearest Neighbors - Cross-Validation Accuracy: {cv_scores.mean()}")


In [ ]:
from sklearn.svm import SVC

# Create and train the model
svm_model = SVC()
svm_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"Support Vector Machine - Training Data Score: {svm_model.score(X_train_ss_scaled, y_train)}")
print(f"Support Vector Machine - Testing Data Score: {svm_model.score(X_test_ss_scaled, y_test)}")


In [ ]:
from sklearn.svm import SVC

# Create and train the model
svm_model = SVC()
svm_model.fit(X_train_ss_scaled, y_train)

# Predict on test set
y_pred = svm_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"Support Vector Machine - Precision: {precision_score(y_test, y_pred)}")
print(f"Support Vector Machine - Recall: {recall_score(y_test, y_pred)}")
print(f"Support Vector Machine - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(svm_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"Support Vector Machine - Cross-Validation Accuracy: {cv_scores.mean()}")


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train the model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"Logistic Regression - Training Data Score: {logistic_regression_model.score(X_train_ss_scaled, y_train)}")
print(f"Logistic Regression - Testing Data Score: {logistic_regression_model.score(X_test_ss_scaled, y_test)}")


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train the model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_ss_scaled, y_train)

# Predict on test set
y_pred = logistic_regression_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"Logistic Regression - Precision: {precision_score(y_test, y_pred)}")
print(f"Logistic Regression - Recall: {recall_score(y_test, y_pred)}")
print(f"Logistic Regression - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(logistic_regression_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"Logistic Regression - Cross-Validation Accuracy: {cv_scores.mean()}")


In [ ]:


# Create and train the model
decision_tree_model = DecisionTreeClassifier(random_state=1)
decision_tree_model.fit(X_train_ss_scaled, y_train)

# Score the model
print(f"Decision Tree - Training Data Score: {decision_tree_model.score(X_train_ss_scaled, y_train)}")
print(f"Decision Tree - Testing Data Score: {decision_tree_model.score(X_test_ss_scaled, y_test)}")


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create and train the model
decision_tree_model = DecisionTreeClassifier(random_state=1)
decision_tree_model.fit(X_train_ss_scaled, y_train)

# Predict on test set
y_pred = decision_tree_model.predict(X_test_ss_scaled)

# Calculate precision, recall, F1 score
print(f"Decision Tree - Precision: {precision_score(y_test, y_pred)}")
print(f"Decision Tree - Recall: {recall_score(y_test, y_pred)}")
print(f"Decision Tree - F1 Score: {f1_score(y_test, y_pred)}")

# Cross-validation scores
cv_scores = cross_val_score(decision_tree_model, X_train_ss_scaled, y_train, cv=5, scoring='accuracy')
print(f"Decision Tree - Cross-Validation Accuracy: {cv_scores.mean()}")
